<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [0]:
from tensorflow.keras.callbacks import LambdaCallback, EarlyStopping, TensorBoard
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

import numpy as np
import random
import sys
import requests
import os
import datetime

In [0]:
url = "https://www.gutenberg.org/files/100/100-0.txt"

r = requests.get(url)

In [4]:
type(r)

requests.models.Response

In [0]:
r.encoding = r.apparent_encoding

data = r.text

In [6]:
data[:100]

'\r\nProject Gutenberg’s The Complete Works of William Shakespeare, by William\r\nShakespeare\r\n\r\nThis eBo'

In [0]:
data = data.split('\r\n')

In [8]:
type(data)

list

In [9]:
len(data)

166903

In [10]:
data[44:48]

['               ALL’S WELL THAT ENDS WELL',
 '',
 '               THE TRAGEDY OF ANTONY AND CLEOPATRA',
 '']

In [39]:
data[2900:2990]

['This brand she quenched in a cool well by,',
 'Which from Love’s fire took heat perpetual,',
 'Growing a bath and healthful remedy,',
 'For men discased, but I my mistress’ thrall,',
 '  Came there for cure and this by that I prove,',
 '  Love’s fire heats water, water cools not love.',
 '',
 '',
 'THE END',
 '',
 '',
 '',
 'ALL’S WELL THAT ENDS WELL',
 '',
 '',
 '',
 'Contents',
 '',
 'ACT I',
 'Scene I. Rossillon. A room in the Countess’s palace.',
 'Scene II. Paris. A room in the King’s palace.',
 'Scene III. Rossillon. A Room in the Palace.',
 '',
 '',
 'ACT II',
 'Scene I. Paris. A room in the King’s palace.',
 'Scene II. Rossillon. A room in the Countess’s palace.',
 'Scene III. Paris. The King’s palace.',
 'Scene IV. Paris. The King’s palace.',
 'Scene V. Another room in the same.',
 '',
 '',
 'ACT III',
 'Scene I. Florence. A room in the Duke’s palace.',
 'Scene II. Rossillon. A room in the Countess’s palace.',
 'Scene III. Florence. Before the Duke’s palace.',
 'Scene IV. Ro

In [40]:
data[30:50]

['',
 '',
 '',
 'by William Shakespeare',
 '',
 '',
 '',
 '',
 '      Contents',
 '',
 '',
 '',
 '               THE SONNETS',
 '',
 '               ALL’S WELL THAT ENDS WELL',
 '',
 '               THE TRAGEDY OF ANTONY AND CLEOPATRA',
 '',
 '               AS YOU LIKE IT',
 '']

In [0]:
# Skip the Table of Contents
theData = data[135:]

In [86]:
# looking at the data so see what it is looking like
theData[39228+2500:(39228+4700)]

['',
 "    Who's here? Westmoreland?",
 '  WESTMORELAND. Health to my sovereign, and new happiness',
 '    Added to that that am to deliver!',
 "    Prince John, your son, doth kiss your Grace's hand.",
 '    Mowbray, the Bishop Scroop, Hastings, and all,',
 '    Are brought to the correction of your law.',
 "    There is not now a rebel's sword unsheath'd,",
 '    But Peace puts forth her olive everywhere.',
 '    The manner how this action hath been borne',
 '    Here at more leisure may your Highness read,',
 '    With every course in his particular.',
 '  KING. O Westmoreland, thou art a summer bird,',
 '    Which ever in the haunch of winter sings',
 '    The lifting up of day.',
 '',
 '                        Enter HARCOURT',
 '',
 "    Look here's more news.",
 '  HARCOURT. From enemies heaven keep your Majesty;',
 '    And, when they stand against you, may they fall',
 '    As those that I am come to tell you of!',
 '    The Earl Northumberland and the Lord Bardolph,',
 '    Wi

In [13]:
# Looking at what the data looks like
theData[:10]

['THE SONNETS',
 '',
 '                    1',
 '',
 'From fairest creatures we desire increase,',
 'That thereby beauty’s rose might never die,',
 'But as the riper should by time decease,',
 'His tender heir might bear his memory:',
 'But thou contracted to thine own bright eyes,',
 'Feed’st thy light’s flame with self-substantial fuel,']

In [0]:
# Had to change this to add the sonnets # made the slice object to inlcude the sonnets
toc = [l.strip() for l in data[42:130:2]]

In [0]:
#THE LIFE OF KING HENRY V
#THE LIFE OF KING HENRY V

In [60]:
toc

['THE SONNETS',
 'ALL’S WELL THAT ENDS WELL',
 'THE TRAGEDY OF ANTONY AND CLEOPATRA',
 'AS YOU LIKE IT',
 'THE COMEDY OF ERRORS',
 'THE TRAGEDY OF CORIOLANUS',
 'CYMBELINE',
 'THE TRAGEDY OF HAMLET, PRINCE OF DENMARK',
 'THE FIRST PART OF KING HENRY THE FOURTH',
 'THE SECOND PART OF KING HENRY THE FOURTH',
 'THE LIFE OF KING HENRY THE FIFTH',
 'THE FIRST PART OF HENRY THE SIXTH',
 'THE SECOND PART OF KING HENRY THE SIXTH',
 'THE THIRD PART OF KING HENRY THE SIXTH',
 'KING HENRY THE EIGHTH',
 'KING JOHN',
 'THE TRAGEDY OF JULIUS CAESAR',
 'THE TRAGEDY OF KING LEAR',
 'LOVE’S LABOUR’S LOST',
 'THE TRAGEDY OF MACBETH',
 'MEASURE FOR MEASURE',
 'THE MERCHANT OF VENICE',
 'THE MERRY WIVES OF WINDSOR',
 'A MIDSUMMER NIGHT’S DREAM',
 'MUCH ADO ABOUT NOTHING',
 'THE TRAGEDY OF OTHELLO, MOOR OF VENICE',
 'PERICLES, PRINCE OF TYRE',
 'KING RICHARD THE SECOND',
 'KING RICHARD THE THIRD',
 'THE TRAGEDY OF ROMEO AND JULIET',
 'THE TAMING OF THE SHREW',
 'THE TEMPEST',
 'THE LIFE OF TIMON OF ATHENS'

In [96]:
# I am changing the correct title for the life of king henry in the toc
toc[10]= "THE LIFE OF KING HENRY V"
toc[10]

'THE LIFE OF KING HENRY V'

In [61]:
len(toc)

44

In [0]:
locations = {id_:{'title':title, 'start':-99} for id_,title in enumerate(toc)}

In [98]:
locations

{0: {'start': -99, 'title': 'THE SONNETS'},
 1: {'start': -99, 'title': 'ALL’S WELL THAT ENDS WELL'},
 2: {'start': -99, 'title': 'THE TRAGEDY OF ANTONY AND CLEOPATRA'},
 3: {'start': -99, 'title': 'AS YOU LIKE IT'},
 4: {'start': -99, 'title': 'THE COMEDY OF ERRORS'},
 5: {'start': -99, 'title': 'THE TRAGEDY OF CORIOLANUS'},
 6: {'start': -99, 'title': 'CYMBELINE'},
 7: {'start': -99, 'title': 'THE TRAGEDY OF HAMLET, PRINCE OF DENMARK'},
 8: {'start': -99, 'title': 'THE FIRST PART OF KING HENRY THE FOURTH'},
 9: {'start': -99, 'title': 'THE SECOND PART OF KING HENRY THE FOURTH'},
 10: {'start': -99, 'title': 'THE LIFE OF KING HENRY V'},
 11: {'start': -99, 'title': 'THE FIRST PART OF HENRY THE SIXTH'},
 12: {'start': -99, 'title': 'THE SECOND PART OF KING HENRY THE SIXTH'},
 13: {'start': -99, 'title': 'THE THIRD PART OF KING HENRY THE SIXTH'},
 14: {'start': -99, 'title': 'KING HENRY THE EIGHTH'},
 15: {'start': -99, 'title': 'KING JOHN'},
 16: {'start': -99, 'title': 'THE TRAGEDY OF

In [89]:
# the locations index was changed becuase I added the sonnets back into the data
locations[10]

{'end': 47572, 'start': -99, 'title': 'THE LIFE OF KING HENRY THE FIFTH'}

In [99]:
# Start 
for e,i in enumerate(theData):
    for t,title in enumerate(toc):
        if title in i:
            locations[t].update({'start':e})
            
# End            
for title in toc:
    
    t = 0
    
    while t < (len(toc)-1):
        print(t)
        end = (locations[t+1]['start']) - 1
        locations[t]['end'] = end
        t += 1

    # Last One
    locations[t]['end'] = len(theData)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1

In [100]:
locations[9]

{'end': 42636,
 'start': 39228,
 'title': 'THE SECOND PART OF KING HENRY THE FOURTH'}

In [101]:
# checking to see if the locations are correct
locations[10]

{'end': 47572, 'start': 42637, 'title': 'THE LIFE OF KING HENRY V'}

In [104]:
theData[47570:47575]

['', '', '', 'THE FIRST PART OF HENRY THE SIXTH', '']

In [0]:
locations

{0: {'title': 'ALL’S WELL THAT ENDS WELL', 'start': 2777, 'end': 7738},
 1: {'title': 'THE TRAGEDY OF ANTONY AND CLEOPATRA',
  'start': 7739,
  'end': 11840},
 2: {'title': 'AS YOU LIKE IT', 'start': 11841, 'end': 14631},
 3: {'title': 'THE COMEDY OF ERRORS', 'start': 14632, 'end': 17832},
 4: {'title': 'THE TRAGEDY OF CORIOLANUS', 'start': 17833, 'end': 27806},
 5: {'title': 'CYMBELINE', 'start': 27807, 'end': 27824},
 6: {'title': 'THE TRAGEDY OF HAMLET, PRINCE OF DENMARK',
  'start': 27825,
  'end': 34511},
 7: {'title': 'THE FIRST PART OF KING HENRY THE FOURTH',
  'start': 34512,
  'end': 39227},
 8: {'title': 'THE SECOND PART OF KING HENRY THE FOURTH',
  'start': 39228,
  'end': -100},
 9: {'title': 'THE LIFE OF KING HENRY V', 'start': -99, 'end': 47572},
 10: {'title': 'THE FIRST PART OF HENRY THE SIXTH',
  'start': 47573,
  'end': 50843},
 11: {'title': 'THE SECOND PART OF KING HENRY THE SIXTH',
  'start': 50844,
  'end': 54331},
 12: {'title': 'THE THIRD PART OF KING HENRY THE 

In [0]:
for e, i in enumerate(data):
    
    if "ALL’S WELL THAT ENDS WELL" in i:
        print(e)

2777


In [0]:
data[0]

'THE SONNETS'

In [0]:
# divide b/w plays and sonets
sonets = data[:2776]
plays = data[2777:]

In [0]:
data[0]

'THE SONNETS'

In [0]:
def long_lines(lst_ln):
    clean = []
    
    for ln in lst_ln: 
        
        if len(ln) == 0:
            pass
        else:
            pct = len(ln.strip(' ')) / len(ln)

            if pct >= .5:
                clean.append(ln.lstrip())

    return clean

In [0]:
# May Not be Needed
#sonets = long_lines(sonets)
#plays = long_lines(plays)

## Word Encoding

This is just a start, and is not complete yet. 

In [0]:
vocab = list(set("\r\n".join(plays).split()))
words = [line.split() for line in plays]

In [0]:
len(vocab)

## Character Encoding

Using the technique shown in lecture. 

In [0]:
text = '\r\n'.join(sonets)

chars = list(set(text))

char_int = {c:i for i,c in enumerate(chars)}
int_char = {i:c for i,c in enumerate(chars)}

print(f"Our corpus contains {len(chars)} unique characters.")

Our corpus contains 73 unique characters.


In [0]:
# Create the Sequence Data

maxlen = 150
step = 1

encoded = [char_int[c] for c in text]

sequences = [] # Each element is 40 characters long
next_chars = [] # One element for each sequence

for i in range(0, len(encoded) - maxlen, step):
    sequences.append(encoded[i : i + maxlen])
    next_chars.append(encoded[i + maxlen])
    
print('sequences:', len(sequences))

sequences: 100978


In [0]:
import numpy as np

# Specify x & y

x = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences), len(chars)), dtype=np.bool)

for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        x[i,t,char] = 1
        
    y[i, next_chars[i]] = 1

In [0]:
x.shape

(100978, 150, 73)

In [0]:
# build the model: a single LSTM

model = Sequential()
model.add(LSTM(256, input_shape=(maxlen, len(chars)), dropout=0.2))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='nadam')

In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 1028)              4531424   
_________________________________________________________________
dense (Dense)                (None, 73)                75117     
Total params: 4,606,541
Trainable params: 4,606,541
Non-trainable params: 0
_________________________________________________________________


In [0]:
def sample(preds):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / 1
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [0]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    
    start_index = random.randint(0, len(text) - maxlen - 1)
    
    generated = ''
    
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)
    
    for i in range(400):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_int[char]] = 1
            
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds)
        next_char = int_char[next_index]
        
        sentence = sentence[1:] + next_char
        
        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [0]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(logdir, histogram_freq=1)

model.fit(x, y,
          batch_size=1024,
          validation_split=.2,
          epochs=100,
          callbacks=[print_callback, 
                     #EarlyStopping(min_delta=.02, monitor='val_loss', patience=10),
                     tensorboard_callback])

Train on 80782 samples, validate on 20196 samples
Epoch 1/100
 2048/80782 [..............................] - ETA: 26:02 - loss: 4.0877
----- Generating text after Epoch: 0
----- Generating with seed: "Each changing place with that which goes before,
In sequent toil all forwards do contend.
Nativity once in the main of light,
Crawls to maturity, w"
Each changing place with that which goes before,
In sequent toil all forwards do contend.
Nativity once in the main of light,
Crawls to maturity, wcyoObdsGhahotnlrior soGdon
diirhma
s2flhEA srSosiiGnohr
 cuish
swhothuhgn
s8i5hplhsahbhv
!hr Phodhho ihrlhnzolhivuoiRr rhf3h
izws
olhaaonhyloAwisellollhhh,lnOt5htrzhylsTnh criYvl1ehyTsaagrwsFchDbriBo:isnhlnotAtOr.rooslatthh,o,S dsf 5ol ohbrlt?(rtronEhnilr5ghc,svrimash,b, ogBaalOxSgochhwthoif!Rn do5wqcgnn at3krwn.
sdssiil,olamrr Tlohg,r
yrOhwioe
wiwJtlfia3soofvadAytaytc p,mhiGo oyvf rtn5h
 2048/80782 [..............................] - ETA: 1:09:39 - loss: 4.0877

KeyboardInterrupt: 

In [0]:
%load_ext tensorboard

In [0]:
%tensorboard --logdir logs

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN